This notebook performs the preprocessing of the raw data.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

import pandas as pd
import numpy as np

from src.preprocess import parse_all_data, clean_raw_data, computeTimeDeltaTraces, computePowerDeltaTraces

import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "none"

import src.colors as colors
from src.helpers import Modes, Parameters
from src.stats import ThompsonCI_twosided
from src.plots import prr_f_TimeDelta, prr_f_PowerDelta, prr_3d

In [4]:
# Clean up and bug correction of the data collected by Anna
# clean_raw_data()

In [5]:
# Create and save the median values for faster plotting of the different settings
# computeTimeDeltaTraces()
# display(df)

# print(sorted(df['PowerDelta'].unique()))
# print(df['PowerDelta'].unique())
# computePowerDeltaTraces()

In [6]:
# pd.read_csv('data_preprocessed/transmitter_pair_a/different_data/PowerDeltaTraces_A_0_(-140).csv')

In [7]:
n_samples = 10
percentile = 50
confidence = 75
ThompsonCI_twosided(n_samples, percentile, confidence)

(2, 7)

In [17]:
# Helix equation
t = np.linspace(0, 10, 50)
x, y, z = np.cos(t), np.sin(t), t
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z,
                                   mode='markers')])
print(go.Scatter3d)

print("\n\nJSON Representation of A Graph Object:\n" + str(fig.to_json()))


<class 'plotly.graph_objs.Scatter3d'>


JSON Representation of A Graph Object:
{"data":[{"mode":"markers","type":"scatter3d","x":[1.0,0.9792475210564969,0.9178514149905888,0.8183599245989672,0.6849024400004521,0.5230181084730104,0.33942593237925484,0.14174589725634046,-0.061817295362854206,-0.262814763741325,-0.4529041164186286,-0.6241957028171256,-0.7695800728569471,-0.883023054382162,-0.95981620122199,-0.996772217050833,-0.9923572439880433,-0.9467545253046645,-0.8618567999191831,-0.7411877443484259,-0.5897557226621226,-0.41384591454310704,-0.22075944916926946,-0.018510372154503518,0.18450697707700792,0.37986637199507933,0.5594594291408053,0.7158321462405541,0.842494280256423,0.9341887246502055,0.9871097053688656,0.9990607393363355,0.9695457993910898,0.8997899018725932,0.7926882623697213,0.6526861299196696,0.48559428713386943,0.29834787391040735,0.09871854474461515,-0.10500809346346798,-0.30437637517455507,-0.4911115282522227,-0.6574631180319587,-0.7965267287855186,-0.9025305312049614

In [36]:
# Compute (or retrieve) all preprocessed data
df = parse_all_data(force_computation='')

# Parameters
PowerDelta = 5
TimeDelta = -140
SamePayload = 0
TransPair = "A"
ModesToShow = [mode for mode in Modes]
# ModesToShow = [ModesToShow[1]]

# print(ModesToShow)

# prr_f_PowerDelta(
#     df,
#     TimeDelta,
#     SamePayload,
#     TransPair,
#     ModesToShow)

fig = prr_3d(
    df,
    SamePayload,
    TransPair,
    ModesToShow)


threeD_plot = go.Figure()
threeD_plot=fig
threeD_plot.show()

Processed data retrieved.


Processed data retrieved.


In [32]:
# Compute (or retrieve) all preprocessed data
df = parse_all_data()
data_path = Path('data_preprocessed')

PowerDelta,SamePayload,TransPair = 9, 1, "B"

filter = (
#     (df['DateTime'] == "2019-10-04 13:55:00") & 
    (df['TransPair'] == "B") & 
    (df['SamePayload'] == 1) & 
    (df["PowerDelta"] == 9)  
)
filtered_df = df.where(filter).dropna()
# display(filtered_df)

Processed data retrieved.


In [60]:
figure = go.Figure()


# Load the median and CI data
file_path = data_path / Parameters['TransPair'][TransPair]['path'] / Parameters['SamePayload'][SamePayload]['path']
file_name = 'TimeDeltaTraces_%s_%s_(%i).csv' % (TransPair,SamePayload,PowerDelta)
df_median = pd.read_csv(file_path / file_name)

for mode in Modes:
    
    mode_filter = (filtered_df["Mode"] == Modes[mode]['id'])
    mode_df = filtered_df.where(mode_filter).dropna()
    
    # Force displaying the trace, even if empty
    if len(mode_df) == 0:
        x_data = [np.nan]
        y_data = [np.nan]
        x_median = [np.nan]
        y_median = [np.nan]
    else: 
        x_data = mode_df["TimeDelta"].to_list()
        y_data = mode_df["PRR"].to_list()
        
        # Extract median data
        x_median = df_median["TimeDelta"].to_list()
        y_median = df_median['median_'+mode].to_list()

        # Extract CI data
        y_LB = df_median['LB_'+mode].to_list()
        y_UB = df_median['UB_'+mode].to_list()
        
        # Prepare the trace for the CI area
        x_CI = x_median + x_median[::-1]
        y_CI = y_LB + y_UB[::-1]
    
    scatter = go.Scatter(
        x=x_data,
        y=y_data,
        mode='markers',
        marker={'color':Modes[mode]['color']},
        showlegend=True,
        legendgroup=Modes[mode]['id'],
        name=Modes[mode]['label'],
        opacity=0.7,
        hoverinfo='skip',
    )
    figure.add_trace(scatter)
    
    
    median_line = go.Scatter(
        x=x_median,
        y=y_median,
        mode='lines',
        line={'color':Modes[mode]['color']},
        showlegend=False,
        legendgroup=Modes[mode]['id'],
        name=Modes[mode]['label']+' median',
    )
    figure.add_trace(median_line)
    
    CI = go.Scatter(
        x=x_CI,
        y=y_CI,
        mode='lines',
        line={
            'color':Modes[mode]['color'],
            'width':0
        },
        showlegend=False,
        legendgroup=Modes[mode]['id'],
        name=Modes[mode]['label']+' CI',
        fill='toself',
        hoverinfo='skip',
        opacity=0.3,
    )
    figure.add_trace(CI)
    
    
    
custom_layout = {
    "title":{
        'text':'PRR = f(Time Delta) <br> with Power Delta = %d dB' % 0,
    },
#     "width":linewidth_px,
#     "height":220,
#     "font":{"size":font_size},
    "margin":dict(t=150),
    "xaxis":{
        'title':{'text':'Transmitters Time Delta [ticks]'},
            },
    "yaxis":{
        'range':[0,103]
    },
    "legend":{
#         'font':{'size':font_size},
        'orientation':"h",
        'x': 0.5,
        'y':1.15,
        'xanchor':"center",
        'yanchor':"top",
    }
}
figure.update_layout(custom_layout)
figure.show()